In [1]:
%run -i bestestimator_v2.py

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [68]:
import pandas as pd
import numpy as np
import operator

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor, \
    RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelBinarizer
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import roc_auc_score


def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return (roc_auc_score(y_test, y_pred, average=average))


class BestEstimator(object):

    def __init__(self, 
                 type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3):
        
        self.type_esti = type_esti
        #self.params = params
        self.cv = cv
        self.grid = grid
        self.hard_grid = hard_grid
        self.cv_grid = cv_grid

        #self.Data = Data.copy()
        #self.Target = Target.copy()
        #self.dim_ = Data.shape
        
        #self.scoring = scoring
        
        self.AUC = make_scorer(multiclass_roc_auc_score)
        
        self.Decision_Function = None

            #self.scoring = make_scorer(multiclass_roc_auc_score)
        

    def fit(self, data, target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 1000,
            value_nan = 0,
            view_nan = True,
           params = False,
           value = 0,
           scoring = 'AUC'):
        
        loss = scoring 
        Data = data.copy()
        Target = target.copy()

        Data.drop([ID], axis=1, inplace=True)
        if target_ID:
            Target.drop([ID], axis=1, inplace=True)

        if view_nan:
            print("Missing Values :\n")

            total = Data.isnull().sum().sort_values(ascending=False)
            percent = (Data.isnull().sum() / Data.isnull().count()).sort_values(ascending=False) * 100
            missing_data = pd.concat([total, percent], axis=1, keys=['Total', '%'])
            print("{} \n".format(missing_data[(percent > 0)]))

        if type(value) == int:
            Data.fillna(value, inplace=True)
            # self.Test.fillna(value, inplace = True)
            # self.Missing_values()

        elif value == 'bfill':
            Data.fillna('bfill', inplace=True)
            # self.Test.fillna('bfill', inplace = True)
            # self.Missing_values()

        elif value == 'ffill':
            Data.fillna('ffill', inplace=True)
            # self.Test.fillna('ffill', inplace = True)
            # self.Missing_values()

        if Data.isnull().any().any() == False:
            print('NaN data filled by {} \n'.format(value))
        else:
            print('Fail to fill NaN data')

        for i in Data.columns:  ###########

            if Data[i].dtype == object:
                encoder = LabelEncoder()
                encoder.fit(list(Data[i]))
                Data[i] = encoder.transform(list(Data[i]))

            if Data[i].dtype == float:
                Data[i] = Data[i].astype('int')

        for i in Target.columns:
            if Target[i].dtype == object:
                le = LabelEncoder()
                le.fit(list(Target[i]))
                Target[i] = le.transform(list(Target[i]))

        X_tr, X_te, Y_tr, Y_te = train_test_split(Data, Target, random_state=0, test_size=1 / 3)

        print('Searching for the best regressor on {} data using {} loss... \n'.format(n, scoring))

        if self.type_esti == 'classifier':

            # print('\n Searching for the best classifier on {} data... \n'.format(n))

            clfs = {}
            clfs['Bagging'] = {'clf': BaggingClassifier(), 'name': 'Bagging'}
            clfs['Gradient Boosting'] = {'clf': GradientBoostingClassifier(), 'name': 'Gradient Boosting'}
            clfs['XGBoost'] = {'clf': XGBClassifier(), 'name': 'XGBoost'}
            clfs['Random Forest'] = {'clf': RandomForestClassifier(n_estimators=100, n_jobs=-1),
                                     'name': 'Random Forest'}
            clfs['Decision Tree'] = {'clf': DecisionTreeClassifier(), 'name': 'Decision Tree'}
            clfs['Extra Tree'] = {'clf': ExtraTreesClassifier(n_jobs=-1), 'name': 'Extra Tree'}

            clfs['KNN'] = {'clf': KNeighborsClassifier(n_jobs=-1), 'name': 'KNN'}
            # clfs['NN'] = {'clf': MLPClassifier(), 'name': 'MLPClassifier'
            # clfs['LR'] = {'clf': LogisticClassifier(), 'name': 'LR'}
            clfs['SVM'] = {'clf': SVC(gamma='auto'), 'name': 'SVM'}

            
            if scoring == 'AUC' and np.unique(Target).shape[0] > 2:
                scoring = self.AUC
                score = 'AUC'
            else :
                score = 'AUC'
                scoring = 'roc_auc'
            
            for item in clfs:
                
                Score = cross_val_score(clfs[item]['clf'], np.asarray(X_tr[0:n]), np.ravel(Y_tr[0:n]),
                                        cv=self.cv, scoring=scoring)
               
                Score_mean = Score.mean()
                STD2 = Score.std() * 2

                clfs[item]['score'] = Score  # roc_auc
                clfs[item]['mean'] = Score_mean
                clfs[item]['std2'] = STD2

                print("\n {}".format(item + ": %0.4f (+/- %0.4f)" % (clfs[item]['score'].mean(),
                                                                     clfs[item]['score'].std() * 2)))

            Best_clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['name']


        elif self.type_esti == 'regressor':

            clfs = {}
            clfs['Bagging'] = {'clf': BaggingRegressor(), 'name': 'Bagging'}
            clfs['Gradient Boosting'] = {'clf': GradientBoostingRegressor(), 'name': 'Gradient Boosting'}
            clfs['XGBoost'] = {'clf': XGBRegressor(), 'name': 'XGBoost'}
            clfs['Random Forest'] = {'clf': RandomForestRegressor(n_estimators=100, n_jobs=-1),
                                     'name': 'Random Forest'}
            clfs['Decision Tree'] = {'clf': DecisionTreeRegressor(), 'name': 'Decision Tree'}
            clfs['Extra Tree'] = {'clf': ExtraTreesRegressor(n_jobs=-1), 'name': 'Extra Tree'}
            clfs['KNN'] = {'clf': KNeighborsRegressor(n_jobs=-1), 'name': 'KNN'}
            # clfs['NN'] = {'clf': MLPClassifier(), 'name': 'MLPClassifier'
            # clfs['LR'] = {'clf': LogisticClassifier(), 'name': 'LR'}
            clfs['SVM'] = {'clf': SVR(gamma='auto'), 'name': 'SVM'}

            for item in clfs:
                # print(Y_tr[0:30])
                Score = cross_val_score(clfs[item]['clf'], np.asarray(X_tr[0:n]), np.array(np.ravel(Y_tr[0:n])),
                                        ########""
                                        cv=self.cv, scoring=scoring)
                Score_mean = Score.mean()
                STD2 = Score.std() * 2

                clfs[item]['score'] = Score  # roc_auc
                clfs[item]['mean'] = Score_mean
                clfs[item]['std2'] = STD2

                print("\n {}".format(item + ": %0.4f (+/- %0.4f)" % (clfs[item]['score'].mean(),
                                                                     clfs[item]['score'].std() * 2)))

            Best_clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['name']

        if self.grid:
            # print('grid = True')

            if params == False:
                # print('params = False')

                # print(Best_clf)

                if self.hard_grid == False:

                    if Best_clf == 'Extra Tree':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600],
                                      'criterion': ['mse', 'mae'],
                                      'max_depth': [None, 5, 10]}

                        else:

                            params = {'n_estimators': [100, 300, 600],
                                      'criterion': ['gini', 'entropy'],
                                      'max_depth': [None, 5, 10]}

                    if Best_clf == 'Gradient Boosting':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600],
                                      'max_depth': [5, 10, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['ls', 'lad']}
                        else:

                            params = {'n_estimators': [100, 300, 600],
                                      'max_depth': [5, 10, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['deviance', 'exponential']}


                    elif Best_clf == 'Random Forest':
                        #  print('Best_clf = dt ou rf')

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300],
                                      'max_depth': [5, 10, None],
                                      'criterion': ['mse', 'mae']}

                        else:

                            params = {'n_estimators': [10, 100, 300],
                                      'max_depth': [5, 10, None],
                                      'criterion': ['gini', 'entropy']}

                    elif Best_clf == 'Decision Tree':

                        if self.type_esti == 'regressor':

                            params = {'max_depth': [5, 10, 50, None],
                                      'criterion': ['mse', 'friedman_mse', 'mae']}

                        else:

                            params = {'max_depth': [5, 10, 50, None],
                                      'criterion': ['gini', 'entropy']}


                    elif Best_clf == 'XGBoost':
                        # print('Best_clf = xgb')

                        params = {'eta': [.01, .1, .3],
                                  'max_depth': [5, 10, 15],
                                  'gamma': [0, .1, .01]}

                    elif Best_clf == 'Bagging':
                        # print('best_clf = bag)')

                        params = {'n_estimators': [100, 300, 600]}

                    elif Best_clf == 'KNN':

                        params = {'n_neighbors': [2, 5, 10, 30, 40],
                                  'p': [1, 2]}

                    elif Best_clf == 'SVM':

                        params = {'C': {1, .5, .1, 5},
                                  'tol': [.01, .001, .1, .0001]}



                else:

                    if Best_clf == 'Extra Tree':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'criterion': ['mae', 'mse'],
                                      'max_depth': [None, 5, 10, 15, 20, 25]}

                        else:

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'criterion': ['gini', 'entropy'],
                                      'max_depth': [None, 5, 10, 15, 20, 25]}

                    if Best_clf == 'Gradient Boosting':

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 25, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['ls', 'lad', 'huber', 'quantile'],
                                      'criterion': ['mse', 'friedman_mse']}
                        else:

                            params = {'n_estimators': [100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 25, None],
                                      'learning_rate': [.001, .01, .1],
                                      'loss': ['deviance', 'exponential'],
                                      'criterion': ['mse', 'friedman_mse']}


                    elif Best_clf == 'Random Forest':
                        #  print('Best_clf = dt ou rf')

                        if self.type_esti == 'regressor':

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 20, 25, None],
                                      'criterion': ['mse', 'mae']}

                        else:

                            params = {'n_estimators': [10, 100, 300, 600, 1000, 1200],
                                      'max_depth': [5, 10, 15, 20, 25],
                                      'criterion': ['gini', 'entropy']}

                    elif Best_clf == 'Decision Tree':

                        if params == 'regressor':

                            params = {'max_depth': [5, 10, 50, 100, None],
                                      'criterion': ['mse', 'friedman_mse', 'mae'],
                                      'splitter': ['best', 'random']}

                        else:

                            params = {'max_depth': [5, 10, 50, 100, None],
                                      'criterion': ['gini', 'entropy'],
                                      'splitter': ['best', 'random']}


                    elif Best_clf == 'XGBoost':
                        # print('Best_clf = xgb')

                        params = {'eta': [0.001, .01, .1, .3, 1],
                                  'max_depth': [5, 10, 15, 20, 25],
                                  'gamma': [0, .1, .01, .001]}

                    elif Best_clf == 'Bagging':
                        # print('best_clf = bag)')

                        params = {'n_estimators': [100, 300, 600, 1000, 1200, 1500]}

                    elif Best_clf == 'KNN':

                        params = {'n_neighbors': [2, 5, 10, 30, 40, 70, 100],
                                  'p': [1, 2, 3]}

                    elif Best_clf == 'SVM':

                        params = {'C': {1, .5, .1, 5, .01, .001},
                                  'tol': [.01, .001, .1, .0001, 1],
                                  'kernel': ['rbf', 'linear', 'poly', 'sigmoid', 'precomputed']}

            if self.hard_grid:
                print('\n Searching for the best hyperparametres of {} using hard_grid on {} data among : \n'.format(
                    Best_clf, n_grid))

            else:
                print('\n Searching for the best hyperparametres of {} on {} data among : \n'.format(Best_clf, n_grid))
            print('{} \n'.format(params))
            # print('Starting GridSearchCV using {} Classifier with {} folds \n'.format(Best_clf, cv_grid))

            
        
            clf = clfs[max(clfs.keys(), key=(lambda k: clfs[k]['mean']))]['clf']

            if loss == 'AUC' and np.unique(Target).shape[0] > 2:
                
                
                gr = GridSearchCV(clf, param_grid=params, cv=self.cv_grid, scoring=scoring, 
                                verbose=1, refit=True, iid=True)
            else :
                gr = GridSearchCV(clf, param_grid=params, cv=self.cv_grid, scoring=scoring, 
                                    verbose=1, refit=True, iid=True, n_jobs = -1)

            gr.fit(X_tr[0:n_grid], np.ravel(Y_tr[0:n_grid]))
            


            # print(' Best score :', gr.best_score_,   '\n Using these parametres :', gr.best_params_)

            #####

            print('\n Finally, the best estimator is : {} {}'.format(Best_clf, self.type_esti))

            print('\n Using these hyperparametres : {}'.format(gr.best_params_))

            print('\n With this {} score : {}'.format(loss, gr.best_score_))
            
            self.Decision_Function = gr.best_estimator_
        else:
            print('\n Best {} : {}'.format(self.type_esti, Best_clf))
            
            
            
    def ReFit(self, Train, Target, ID = 'ID', target_ID = 'ID', value = 0):
        
        train = Train.copy()
        target = Target.copy()
        
        train = self.Transform(train, value = value, ID = ID)
        target = self.Transform(target, value = value, ID = ID)
        
        estim = self.Decision_Function.fit(train, target)
        
        return(estim)

    
    
    def Transform(self, Data, value=0, ID = 'ID'):
        
        Test = Data.copy()
        
        if ID != None:

            Test.drop([ID], axis=1, inplace=True)

        if type(value) == int:
            Test.fillna(value, inplace=True)

        elif value == 'bfill':
            Test.fillna('bfill', inplace=True)

        elif value == 'ffill':
            Test.fillna('ffill', inplace=True)

        for i in Test.columns:  ###########
            if Test[i].dtype == float:
                Test[i] = Test[i].astype('int')

            elif Test[i].dtype == object:
                encoder = LabelEncoder()
                encoder.fit(list(Test[i]))
                Test[i] = encoder.transform(list(Test[i]))
        return(Test)
    
    
    
    def pred(self, Test, ID = 'ID', value = 0, prob = False):
        pass


    def pred(self, Test, gr, prob=False, same=True, ID='ID', value=0):  #

        # Test.drop([ID], axis = 1, inplace = True)
        Pred = pd.DataFrame()

        if same == False:

            Test.drop([ID], axis=1, inplace=True)

            if type(value) == int:
                Test.fillna(value, inplace=True)

            elif value == 'bfill':
                Test.fillna('bfill', inplace=True)

            elif value == 'ffill':
                Test.fillna('ffill', inplace=True)

            for i in Test.columns:
                if Test[i].dtype == float:
                    Test[i] = Test[i].astype('int')

                elif Test[i].dtype == object:
                    encoder = LabelEncoder()
                    encoder.fit(list(Test[i]))
                    Test[i] = encoder.transform(list(Test[i]))

        if prob == False:
            # Pred[ID] = Test[ID]
            Pred['Target'] = gr.predict(Test)
            return (Pred)

        else:
            return (gr.predict_proba(Test))
        

    def grid(self, clf, params, cv=3, n=100000):

        X_tr, X_te, Y_tr, Y_te = train_test_split(self.Data, self.Target, random_state=0, test_size=1 / 3)

        gr = GridSearchCV(clf, param_grid=params, cv=cv, scoring=self.AUC, n_jobs=-1,
                          verbose=1, refit=True, iid=True);

        gr.fit(X_tr[0:n], np.ravel(Y_tr[0:n]))

        # print(' Best score :', gr.best_score_,   '\n Using this parametres :', gr.best_params_, '\n With :', clf)
        print(' Best score on Train:', gr.best_score_, '\n Using this parametres :', gr.best_params_,
              '\n With : \n {} '.format(clf))
        return gr







### Multiclass

In [39]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train.csv", sep = ',')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target.csv", sep = ';')

### Binary

In [69]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train1.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target1.csv", sep = ';')

### Regression

In [46]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\input_training.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target_Engie.csv", sep = ';')

In [ ]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 500,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'AUC') 

Missing Values :

                            Total         %
1_diffClosing stocks(kmt)     261  2.569151
2_diffClosing stocks(kmt)     257  2.529777
3_diffClosing stocks(kmt)     253  2.490403
4_diffClosing stocks(kmt)     249  2.451029
5_diffClosing stocks(kmt)     245  2.411655
6_diffClosing stocks(kmt)     241  2.372281
7_diffClosing stocks(kmt)     238  2.342750
8_diffClosing stocks(kmt)     235  2.313220
9_diffClosing stocks(kmt)     232  2.283689
10_diffClosing stocks(kmt)    228  2.244315
11_diffClosing stocks(kmt)    224  2.204941
12_diffClosing stocks(kmt)    220  2.165567
1_diffImports(kmt)            151  1.486367
2_diffImports(kmt)            148  1.456836
3_diffImports(kmt)            145  1.427306
4_diffImports(kmt)            142  1.397775
5_diffImports(kmt)            139  1.368245
6_diffImports(kmt)            136  1.338714
7_diffImports(kmt)            133  1.309184
8_diffImports(kmt)            130  1.279654
9_diffImports(kmt)            127  1.250123
10_diffImports

In [36]:
Train = clf.Transform(Train, ID = 'ID', value = 0)  
Target = clf.Transform(Target, ID = 'ID', value = 0)  

In [43]:
A = clf.ReFit(Train[0:50], Target[0:50], ID = 'ID', target_ID = 'ID', value = 0)

In [45]:
A

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [47]:
clf = BestEstimator(type_esti = 'regressor', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = False,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 1000,
            n_grid = 500,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'r2') 

Missing Values :

                                Total          %
Grid_voltage                   101322  16.411451
Grid_voltage_std               101322  16.411451
Grid_voltage_max               101322  16.411451
Grid_voltage_min               101322  16.411451
Gearbox_inlet_temperature        8064   1.306152
Generator_converter_speed        8064   1.306152
Generator_converter_speed_min    8064   1.306152
Generator_converter_speed_max    8064   1.306152
Generator_converter_speed_std    8064   1.306152
Gearbox_inlet_temperature_min    8064   1.306152
Gearbox_inlet_temperature_max    8064   1.306152
Gearbox_inlet_temperature_std    8064   1.306152
Absolute_wind_direction_c          72   0.011662
Nacelle_angle_c                    72   0.011662 

NaN data filled by 0 

Searching for the best regressor on 1000 data using r2 loss... 


 Bagging: 0.9723 (+/- 0.0135)

 Gradient Boosting: 0.9756 (+/- 0.0125)

 XGBoost: 0.9761 (+/- 0.0140)

 Random Forest: 0.9700 (+/- 0.0204)

 Decision Tree: 

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   33.8s finished



 Finally, the best estimator is : XGBoost regressor

 Using these hyperparametres : {'eta': 0.01, 'gamma': 0.01, 'max_depth': 5}

 With this r2 score : 0.958793882476215


In [48]:
A = clf.ReFit(Train[0:50], Target[0:50], ID = 'ID', target_ID = 'ID', value = 0)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [62]:
gr = GridSearchCV(param_grid = {'n_estimators': [300],
                                      'max_depth': [5, 10],
                                      'criterion': ['mse', 'mae']},
                  estimator = RandomForestRegressor()
)

In [63]:
gr.fit(clf.Transform(Train[0:100], ID = 'ID', value=0), clf.Transform(Target[0:100], ID ='ID'))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [300], 'max_depth': [5, 10], 'criterion': ['mse', 'mae']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [65]:
gr.get_params().transform(Train)

AttributeError: 'dict' object has no attribute 'transform'

In [67]:
gr.best_estimator_.fit(Train,Target)

ValueError: could not convert string to float: 'WT1'